# IMDB XGBoost Total Score
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Importing so that I can use LinearRegression and OLS Models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

In [2]:
totalscore_df = pd.read_csv('../data/totalscore_df.csv')

In [3]:
totalscore_df.columns

Index(['movie_title', 'year', 'actors', 'plot', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Thriller', 'War', 'Western', 'avg_vote', 'votes',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating',
       'us_voters_votes', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score', 'total_score'],
      dtype='object')

### XGBoost

In [4]:
from xgboost import XGBRegressor, XGBClassifier

In [5]:
xgb = XGBRegressor()

In [6]:
features = ['duration', 'year', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Horror', 'votes_1',
            'votes_4',  'votes_6', 'votes_10', 'director_score', 'actor_score', 'actress_score']

In [7]:
X = totalscore_df[features]
y = totalscore_df['total_score']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [9]:
xgb.fit(X_train, y_train);

In [10]:
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

Training Score: 0.9864547298466866
 
Testing Score: 0.9400197744710536


In [11]:
predictions = xgb.predict(X_train)

residuals = y_train - predictions

rss = (residuals ** 2).sum()

mse = metrics.mean_squared_error(y_train, predictions)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))

In [12]:
null_predictions = np.zeros_like(y_test)

null_pred = null_predictions + y_test.mean()

baseline = metrics.mean_squared_error(y_test, null_pred, squared = False)

In [13]:
print("Baseline RMSE:", baseline)
print(" ")
print("RMSE:", rmse)
print("________________________________")
print(" ")
print("Difference:", baseline - rmse)

Baseline RMSE: 1.2910207205254103
 
RMSE: 0.15440657130495902
________________________________
 
Difference: 1.1366141492204513


In [14]:
print("RMSE:", rmse)
print(" ")
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

RMSE: 0.15440657130495902
 
Training Score: 0.9864547298466866
 
Testing Score: 0.9400197744710536


## GridSearch

In [15]:
xgb = XGBRegressor()

In [16]:
model_params = {}
count = 0

In [22]:
xgb_params = {
    'base_score' : [.15, .20],
    'booster' : ['gbtree', 'gblinear', 'dart'],
    'learning_rate': [.15, .20],
    'max_depth' : [2, 3],
    'n_estimators': [800, 900, 1000],
}

gs = GridSearchCV(xgb, param_grid = xgb_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient = 'index')

model_df

,base_score,booster,learning_rate,max_depth,n_estimators,best_score,training_score,testing_score
model_1,0.15,dart,0.20,2,300,0.941992,0.954136,0.940976
model_2,0.10,gbtree,0.15,3,400,0.950789,0.969751,0.949735
model_3,0.15,dart,0.15,3,500,0.951671,0.972121,0.951013
model_4,0.20,dart,0.15,3,600,0.952283,0.973967,0.951754
model_5,0.20,gbtree,0.15,3,800,0.952811,0.977009,0.952606
model_6,0.20,gbtree,0.15,3,1000,0.953032,0.979414,0.952972
